In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# Define all possible features
all_features = ['rate_level_1', 'rate_level_2', 'days_to_end_of_month', 'days_to_ECB_meeting', 'days_to_Fed_meeting', 'ois_sofr_rate',
                'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15']

# Function to split list into chunks
def chunk_list(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# Split the features into 3 columns
num_columns = 3
chunks = list(chunk_list(all_features, len(all_features) // num_columns + 1))

# Create checkboxes for each feature
checkboxes = [[widgets.Checkbox(value=True, description=feature) for feature in chunk] for chunk in chunks]

# Create a grid layout for the checkboxes
grid = widgets.GridBox([item for sublist in checkboxes for item in sublist],
                       layout=widgets.Layout(grid_template_columns=f'repeat({num_columns}, 200px)'))

# Function to get selected features
def get_selected_features():
    return [checkbox.description for sublist in checkboxes for checkbox in sublist if checkbox.value]

# Add a button to confirm feature selection
button_features = widgets.Button(description="Confirm Features")
output_features = widgets.Output()

# Global variable to store selected features
selected_features = []

def on_button_clicked_features(b):
    global selected_features
    with output_features:
        selected_features = get_selected_features()
        print("Selected features:", selected_features)

button_features.on_click(on_button_clicked_features)

# Create widgets for hyperparameters with adjusted layout
layout = widgets.Layout(width='400px')  # Adjust the width of the description area

learning_rate_slider = widgets.FloatSlider(
    value=0.001,
    min=0.0001,
    max=0.01,
    step=0.00001,  # Increase precision
    description='Learning Rate:',
    continuous_update=False,
    readout_format='.5f',  # Display more decimal places
    style={'description_width': 'initial'},
    layout=layout
)

n_epochs_slider = widgets.IntSlider(
    value=50,
    min=10,
    max=200,
    step=10,
    description='Epochs:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=layout
)

hidden_size_slider = widgets.IntSlider(
    value=128,
    min=32,
    max=512,
    step=32,
    description='Hidden Size:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=layout
)

dropout_slider = widgets.FloatSlider(
    value=0.3,
    min=0.0,
    max=0.5,
    step=0.1,
    description='Dropout Rate:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=layout
)

# Display the widgets for hyperparameters
hyperparameter_selection = widgets.VBox([
    learning_rate_slider,
    n_epochs_slider,
    hidden_size_slider,
    dropout_slider
])

# Add a button to confirm hyperparameters
button_hyperparams = widgets.Button(description="Confirm Hyperparameters")
output_hyperparams = widgets.Output()

def on_button_clicked_hyperparams(b):
    with output_hyperparams:
        learning_rate = learning_rate_slider.value
        n_epochs = n_epochs_slider.value
        hidden_size = hidden_size_slider.value
        dropout = dropout_slider.value
        
        print(f"Selected Hyperparameters:\nLearning Rate: {learning_rate:.5f}\nEpochs: {n_epochs}\nHidden Size: {hidden_size}\nDropout Rate: {dropout}")

button_hyperparams.on_click(on_button_clicked_hyperparams)

# Create checkboxes for model selection
lstm_checkbox = widgets.Checkbox(value=False, description='LSTM', style={'description_width': 'initial'}, layout=layout)
momentum_checkbox = widgets.Checkbox(value=False, description='Momentum Strategy', style={'description_width': 'initial'}, layout=layout)
moving_average_checkbox = widgets.Checkbox(value=False, description='Moving Average', style={'description_width': 'initial'}, layout=layout)

# Create checkboxes for LSTM options
attention_checkbox = widgets.Checkbox(value=False, description='Use Attention', style={'description_width': 'initial'}, layout=layout)
autoencoder_checkbox = widgets.Checkbox(value=False, description='Use Autoencoder', style={'description_width': 'initial'}, layout=layout)

# Create a VBox for LSTM-specific options
lstm_options = widgets.VBox([attention_checkbox, autoencoder_checkbox])

# Function to update visibility of LSTM-specific options
def update_lstm_options(change):
    if lstm_checkbox.value:
        lstm_options.layout.display = 'block'
    else:
        lstm_options.layout.display = 'none'

lstm_checkbox.observe(update_lstm_options, names='value')
update_lstm_options(None)  # Initial call to set visibility

# Display the model selection widgets
model_selection = widgets.VBox([
    lstm_checkbox,
    lstm_options,
    momentum_checkbox,
    moving_average_checkbox
])

# Add a button to confirm model selection
button_model = widgets.Button(description="Confirm Model")
output_model = widgets.Output()

def on_button_clicked_model(b):
    with output_model:
        output_model.clear_output()
        selected_models = []
        if lstm_checkbox.value:
            selected_models.append('LSTM')
        if momentum_checkbox.value:
            selected_models.append('Momentum Strategy')
        if moving_average_checkbox.value:
            selected_models.append('Moving Average')
        
        use_attention = attention_checkbox.value if lstm_checkbox.value else None
        use_autoencoder = autoencoder_checkbox.value if lstm_checkbox.value else None
        
        print(f"Selected Models: {selected_models}\nUse Attention: {use_attention}\nUse Autoencoder: {use_autoencoder}")

button_model.on_click(on_button_clicked_model)

# Display all sections in a structured layout
display(widgets.VBox([
    widgets.HTML("<h2>Feature Selection</h2>"),
    grid,
    button_features,
    output_features,
    widgets.HTML("<h2>Hyperparameter Selection</h2>"),
    hyperparameter_selection,
    button_hyperparams,
    output_hyperparams,
    widgets.HTML("<h2>Model Selection</h2>"),
    model_selection,
    button_model,
    output_model
]))

In [32]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# Global variables to store selected values
selected_features = []
selected_hyperparameters = {}
selected_models = []

def interactive_selection():
    global selected_features, selected_hyperparameters, selected_models

    # Define all possible features
    all_features = ['rate_level_1', 'rate_level_2', 'days_to_end_of_month', 'days_to_ECB_meeting', 'days_to_Fed_meeting', 'ois_sofr_rate',
                    'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15']

    # Function to split list into chunks
    def chunk_list(lst, n):
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    # Split the features into 3 columns
    num_columns = 3
    chunks = list(chunk_list(all_features, len(all_features) // num_columns + 1))

    # Create checkboxes for each feature
    checkboxes = [[widgets.Checkbox(value=True, description=feature) for feature in chunk] for chunk in chunks]

    # Create a grid layout for the checkboxes
    grid = widgets.GridBox([item for sublist in checkboxes for item in sublist],
                           layout=widgets.Layout(grid_template_columns=f'repeat({num_columns}, 200px)'))

    # Function to get selected features
    def get_selected_features():
        return [checkbox.description for sublist in checkboxes for checkbox in sublist if checkbox.value]

    # Add buttons to select/deselect all features
    button_select_all_features = widgets.Button(description="Select All Features")
    button_deselect_all_features = widgets.Button(description="Deselect All Features")

    def on_button_select_all_features(b):
        for sublist in checkboxes:
            for checkbox in sublist:
                checkbox.value = True

    def on_button_deselect_all_features(b):
        for sublist in checkboxes:
            for checkbox in sublist:
                checkbox.value = False

    button_select_all_features.on_click(on_button_select_all_features)
    button_deselect_all_features.on_click(on_button_deselect_all_features)

    # Add a button to confirm feature selection
    button_features = widgets.Button(description="Confirm Features")
    output_features = widgets.Output()

    def on_button_clicked_features(b):
        global selected_features
        with output_features:
            selected_features = get_selected_features()
            output_features.clear_output()
            print("Selected features:", selected_features)

    button_features.on_click(on_button_clicked_features)

    # Create widgets for hyperparameters with adjusted layout
    layout = widgets.Layout(width='400px')  # Adjust the width of the description area

    learning_rate_slider = widgets.FloatSlider(
        value=0.001,
        min=0.0001,
        max=0.01,
        step=0.00001,  # Increase precision
        description='Learning Rate:',
        continuous_update=False,
        readout_format='.5f',  # Display more decimal places
        style={'description_width': 'initial'},
        layout=layout
    )

    n_epochs_slider = widgets.IntSlider(
        value=50,
        min=10,
        max=200,
        step=10,
        description='Epochs:',
        continuous_update=False,
        style={'description_width': 'initial'},
        layout=layout
    )

    hidden_size_slider = widgets.IntSlider(
        value=128,
        min=32,
        max=512,
        step=32,
        description='Hidden Size:',
        continuous_update=False,
        style={'description_width': 'initial'},
        layout=layout
    )

    dropout_slider = widgets.FloatSlider(
        value=0.3,
        min=0.0,
        max=0.5,
        step=0.1,
        description='Dropout Rate:',
        continuous_update=False,
        style={'description_width': 'initial'},
        layout=layout
    )

    # Display the widgets for hyperparameters
    hyperparameter_selection = widgets.VBox([
        learning_rate_slider,
        n_epochs_slider,
        hidden_size_slider,
        dropout_slider
    ])

    # Add a button to confirm hyperparameters
    button_hyperparams = widgets.Button(description="Confirm Hyperparameters")
    output_hyperparams = widgets.Output()

    def on_button_clicked_hyperparams(b):
        global selected_hyperparameters
        with output_hyperparams:
            selected_hyperparameters = {
                'learning_rate': learning_rate_slider.value,
                'n_epochs': n_epochs_slider.value,
                'hidden_size': hidden_size_slider.value,
                'dropout': dropout_slider.value
            }
            output_hyperparams.clear_output()
            print(f"Selected Hyperparameters:\nLearning Rate: {selected_hyperparameters['learning_rate']:.5f}\nEpochs: {selected_hyperparameters['n_epochs']}\nHidden Size: {selected_hyperparameters['hidden_size']}\nDropout Rate: {selected_hyperparameters['dropout']}")

    button_hyperparams.on_click(on_button_clicked_hyperparams)

    # Create checkboxes for model selection
    lstm_checkbox = widgets.Checkbox(value=False, description='LSTM', style={'description_width': 'initial'}, layout=layout)
    momentum_checkbox = widgets.Checkbox(value=False, description='Momentum Strategy', style={'description_width': 'initial'}, layout=layout)
    moving_average_checkbox = widgets.Checkbox(value=False, description='Moving Average', style={'description_width': 'initial'}, layout=layout)

    # Create a list of model checkboxes
    model_checkboxes = [lstm_checkbox, momentum_checkbox, moving_average_checkbox]

    # Add buttons to select/deselect all models
    button_select_all_models = widgets.Button(description="Select All Models")
    button_deselect_all_models = widgets.Button(description="Deselect All Models")

    def on_button_select_all_models(b):
        for checkbox in model_checkboxes:
            checkbox.value = True

    def on_button_deselect_all_models(b):
        for checkbox in model_checkboxes:
            checkbox.value = False

    button_select_all_models.on_click(on_button_select_all_models)
    button_deselect_all_models.on_click(on_button_deselect_all_models)

    # Display the model selection widgets
    model_selection = widgets.VBox(model_checkboxes)

    # Add a button to confirm model selection
    button_model = widgets.Button(description="Confirm Model")
    output_model = widgets.Output()

    def on_button_clicked_model(b):
        global selected_models
        with output_model:
            output_model.clear_output()
            selected_models = [checkbox.description for checkbox in model_checkboxes if checkbox.value]
            print(f"Selected Models: {selected_models}")

    button_model.on_click(on_button_clicked_model)

    # Display all sections in a structured layout
    display(widgets.VBox([
        widgets.HTML("<h2>Feature Selection</h2>"),
        grid,
        widgets.HBox([button_select_all_features, button_deselect_all_features]),
        button_features,
        output_features,
        widgets.HTML("<h2>Hyperparameter Selection</h2>"),
        hyperparameter_selection,
        button_hyperparams,
        output_hyperparams,
        widgets.HTML("<h2>Model Selection</h2>"),
        model_selection,
        widgets.HBox([button_select_all_models, button_deselect_all_models]),
        button_model,
        output_model
    ]))

# Call the function to display the widgets
interactive_selection()

In [ ]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import json

# Global variables to store selected values
selected_features = []
selected_hyperparameters = {}
selected_models = []

def save_selected_parameters():
    global selected_features, selected_hyperparameters, selected_models
    parameters = {
        'selected_features': selected_features,
        'selected_hyperparameters': selected_hyperparameters,
        'selected_models': selected_models
    }
    with open('selected_parameters.json', 'w') as f:
        json.dump(parameters, f)

def interactive_selection():
    global selected_features, selected_hyperparameters, selected_models

    # Define all possible features
    all_features = ['rate_level_1', 'rate_level_2', 'days_to_end_of_month', 'days_to_ECB_meeting', 'days_to_Fed_meeting', 'ois_sofr_rate',
                    'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15']

    # Function to split list into chunks
    def chunk_list(lst, n):
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    # Split the features into 3 columns
    num_columns = 3
    chunks = list(chunk_list(all_features, len(all_features) // num_columns + 1))

    # Create checkboxes for each feature
    checkboxes = [[widgets.Checkbox(value=True, description=feature) for feature in chunk] for chunk in chunks]

    # Create a grid layout for the checkboxes
    grid = widgets.GridBox([item for sublist in checkboxes for item in sublist],
                           layout=widgets.Layout(grid_template_columns=f'repeat({num_columns}, 200px)'))

    # Function to get selected features
    def get_selected_features():
        return [checkbox.description for sublist in checkboxes for checkbox in sublist if checkbox.value]

    # Add buttons to select/deselect all features
    button_select_all_features = widgets.Button(description="Select All Features")
    button_deselect_all_features = widgets.Button(description="Deselect All Features")

    def on_button_select_all_features(b):
        for sublist in checkboxes:
            for checkbox in sublist:
                checkbox.value = True

    def on_button_deselect_all_features(b):
        for sublist in checkboxes:
            for checkbox in sublist:
                checkbox.value = False

    button_select_all_features.on_click(on_button_select_all_features)
    button_deselect_all_features.on_click(on_button_deselect_all_features)

    # Add a button to confirm feature selection
    button_features = widgets.Button(description="Confirm Features")
    output_features = widgets.Output()

    def on_button_clicked_features(b):
        global selected_features
        with output_features:
            selected_features = get_selected_features()
            output_features.clear_output()
            print("Selected features:", selected_features)
            save_selected_parameters()

    button_features.on_click(on_button_clicked_features)

    # Create widgets for hyperparameters with adjusted layout
    layout = widgets.Layout(width='400px')  # Adjust the width of the description area

    learning_rate_slider = widgets.FloatSlider(
        value=0.001,
        min=0.0001,
        max=0.01,
        step=0.00001,  # Increase precision
        description='Learning Rate:',
        continuous_update=False,
        readout_format='.5f',  # Display more decimal places
        style={'description_width': 'initial'},
        layout=layout
    )

    n_epochs_slider = widgets.IntSlider(
        value=50,
        min=10,
        max=200,
        step=10,
        description='Epochs:',
        continuous_update=False,
        style={'description_width': 'initial'},
        layout=layout
    )

    hidden_size_slider = widgets.IntSlider(
        value=128,
        min=32,
        max=512,
        step=32,
        description='Hidden Size:',
        continuous_update=False,
        style={'description_width': 'initial'},
        layout=layout
    )

    dropout_slider = widgets.FloatSlider(
        value=0.3,
        min=0.0,
        max=0.5,
        step=0.1,
        description='Dropout Rate:',
        continuous_update=False,
        style={'description_width': 'initial'},
        layout=layout
    )

    # Display the widgets for hyperparameters
    hyperparameter_selection = widgets.VBox([
        learning_rate_slider,
        n_epochs_slider,
        hidden_size_slider,
        dropout_slider
    ])

    # Add a button to confirm hyperparameters
    button_hyperparams = widgets.Button(description="Confirm Hyperparameters")
    output_hyperparams = widgets.Output()

    def on_button_clicked_hyperparams(b):
        global selected_hyperparameters
        with output_hyperparams:
            selected_hyperparameters = {
                'learning_rate': learning_rate_slider.value,
                'n_epochs': n_epochs_slider.value,
                'hidden_size': hidden_size_slider.value,
                'dropout': dropout_slider.value
            }
            output_hyperparams.clear_output()
            print(f"Selected Hyperparameters:\nLearning Rate: {selected_hyperparameters['learning_rate']:.5f}\nEpochs: {selected_hyperparameters['n_epochs']}\nHidden Size: {selected_hyperparameters['hidden_size']}\nDropout Rate: {selected_hyperparameters['dropout']}")
            save_selected_parameters()

    button_hyperparams.on_click(on_button_clicked_hyperparams)

    # Create checkboxes for model selection
    lstm_checkbox = widgets.Checkbox(value=False, description='LSTM', style={'description_width': 'initial'}, layout=layout)
    momentum_checkbox = widgets.Checkbox(value=False, description='Momentum Strategy', style={'description_width': 'initial'}, layout=layout)
    moving_average_checkbox = widgets.Checkbox(value=False, description='Moving Average', style={'description_width': 'initial'}, layout=layout)

    # Create a list of model checkboxes
    model_checkboxes = [lstm_checkbox, momentum_checkbox, moving_average_checkbox]

    # Add buttons to select/deselect all models
    button_select_all_models = widgets.Button(description="Select All Models")
    button_deselect_all_models = widgets.Button(description="Deselect All Models")

    def on_button_select_all_models(b):
        for checkbox in model_checkboxes:
            checkbox.value = True

    def on_button_deselect_all_models(b):
        for checkbox in model_checkboxes:
            checkbox.value = False

    button_select_all_models.on_click(on_button_select_all_models)
    button_deselect_all_models.on_click(on_button_deselect_all_models)

    # Display the model selection widgets
    model_selection = widgets.VBox(model_checkboxes)

    # Add a button to confirm model selection
    button_model = widgets.Button(description="Confirm Model")
    output_model = widgets.Output()

    def on_button_clicked_model(b):
        global selected_models
        with output_model:
            output_model.clear_output()
            selected_models = [checkbox.description for checkbox in model_checkboxes if checkbox.value]
            print(f"Selected Models: {selected_models}")
            save_selected_parameters()

    button_model.on_click(on_button_clicked_model)

    # Display all sections in a structured layout
    display(widgets.VBox([
        widgets.HTML("<h2>Feature Selection</h2>"),
        grid,
        widgets.HBox([button_select_all_features, button_deselect_all_features]),
        button_features,
        output_features,
        widgets.HTML("<h2>Hyperparameter Selection</h2>"),
        hyperparameter_selection,
        button_hyperparams,
        output_hyperparams,
        widgets.HTML("<h2>Model Selection</h2>"),
        model_selection,
        widgets.HBox([button_select_all_models, button_deselect_all_models]),
        button_model,
        output_model
    ]))

def load_selected_parameters():
    global selected_features, selected_hyperparameters, selected_models
    with open('selected_parameters.json', 'r') as f:
        parameters = json.load(f)
        selected_features = parameters['selected_features']
        selected_hyperparameters = parameters['selected_hyperparameters']
        selected_models = parameters['selected_models']

In [ ]:
# Import the module
import interactive_selection as isel

# Load the selected parameters
isel.load_selected_parameters()

# Access the selected parameters
print("Selected Features:", isel.selected_features)
print("Selected Hyperparameters:", isel.selected_hyperparameters)
print("Selected Models:", isel.selected_models)

In [33]:
selected_models

['LSTM']

In [4]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# Define all possible features
all_features = ['rate_level_1', 'rate_level_2', 'days_to_end_of_month', 'days_to_ECB_meeting', 'days_to_Fed_meeting', 'ois_sofr_rate',
                'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15']

# Function to split list into chunks
def chunk_list(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# Split the features into 3 columns
num_columns = 3
chunks = list(chunk_list(all_features, len(all_features) // num_columns + 1))

# Create checkboxes for each feature
checkboxes = [[widgets.Checkbox(value=True, description=feature) for feature in chunk] for chunk in chunks]

# Create a grid layout for the checkboxes
grid = widgets.GridBox([item for sublist in checkboxes for item in sublist],
                       layout=widgets.Layout(grid_template_columns=f'repeat({num_columns}, 200px)'))

# Function to get selected features
def get_selected_features():
    return [checkbox.description for sublist in checkboxes for checkbox in sublist if checkbox.value]

# Add buttons to select/deselect all features
button_select_all_features = widgets.Button(description="Select All Features")
button_deselect_all_features = widgets.Button(description="Deselect All Features")

def on_button_select_all_features(b):
    for sublist in checkboxes:
        for checkbox in sublist:
            checkbox.value = True

def on_button_deselect_all_features(b):
    for sublist in checkboxes:
        for checkbox in sublist:
            checkbox.value = False

button_select_all_features.on_click(on_button_select_all_features)
button_deselect_all_features.on_click(on_button_deselect_all_features)

# Add a button to confirm feature selection
button_features = widgets.Button(description="Confirm Features")
output_features = widgets.Output()

# Global variable to store selected features
selected_features = []

def on_button_clicked_features(b):
    global selected_features
    with output_features:
        selected_features = get_selected_features()
        print("Selected features:", selected_features)

button_features.on_click(on_button_clicked_features)

# Create widgets for hyperparameters with adjusted layout
layout = widgets.Layout(width='400px')  # Adjust the width of the description area

learning_rate_slider = widgets.FloatSlider(
    value=0.001,
    min=0.0001,
    max=0.01,
    step=0.00001,  # Increase precision
    description='Learning Rate:',
    continuous_update=False,
    readout_format='.5f',  # Display more decimal places
    style={'description_width': 'initial'},
    layout=layout
)

n_epochs_slider = widgets.IntSlider(
    value=50,
    min=10,
    max=200,
    step=10,
    description='Epochs:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=layout
)

hidden_size_slider = widgets.IntSlider(
    value=128,
    min=32,
    max=512,
    step=32,
    description='Hidden Size:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=layout
)

dropout_slider = widgets.FloatSlider(
    value=0.3,
    min=0.0,
    max=0.5,
    step=0.1,
    description='Dropout Rate:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=layout
)

# Display the widgets for hyperparameters
hyperparameter_selection = widgets.VBox([
    learning_rate_slider,
    n_epochs_slider,
    hidden_size_slider,
    dropout_slider
])

# Add a button to confirm hyperparameters
button_hyperparams = widgets.Button(description="Confirm Hyperparameters")
output_hyperparams = widgets.Output()

def on_button_clicked_hyperparams(b):
    with output_hyperparams:
        learning_rate = learning_rate_slider.value
        n_epochs = n_epochs_slider.value
        hidden_size = hidden_size_slider.value
        dropout = dropout_slider.value
        
        print(f"Selected Hyperparameters:\nLearning Rate: {learning_rate:.5f}\nEpochs: {n_epochs}\nHidden Size: {hidden_size}\nDropout Rate: {dropout}")

button_hyperparams.on_click(on_button_clicked_hyperparams)

# Create checkboxes for model selection
lstm_checkbox = widgets.Checkbox(value=False, description='LSTM', style={'description_width': 'initial'}, layout=layout)
momentum_checkbox = widgets.Checkbox(value=False, description='Momentum Strategy', style={'description_width': 'initial'}, layout=layout)
moving_average_checkbox = widgets.Checkbox(value=False, description='Moving Average', style={'description_width': 'initial'}, layout=layout)

# Create a list of model checkboxes
model_checkboxes = [lstm_checkbox, momentum_checkbox, moving_average_checkbox]

# Add buttons to select/deselect all models
button_select_all_models = widgets.Button(description="Select All Models")
button_deselect_all_models = widgets.Button(description="Deselect All Models")

def on_button_select_all_models(b):
    for checkbox in model_checkboxes:
        checkbox.value = True

def on_button_deselect_all_models(b):
    for checkbox in model_checkboxes:
        checkbox.value = False

button_select_all_models.on_click(on_button_select_all_models)
button_deselect_all_models.on_click(on_button_deselect_all_models)

# Display the model selection widgets
model_selection = widgets.VBox([
    lstm_checkbox,
    momentum_checkbox,
    moving_average_checkbox
])

# Add a button to confirm model selection
button_model = widgets.Button(description="Confirm Model")
output_model = widgets.Output()

def on_button_clicked_model(b):
    with output_model:
        output_model.clear_output()
        selected_models = []
        if lstm_checkbox.value:
            selected_models.append('LSTM')
        if momentum_checkbox.value:
            selected_models.append('Momentum Strategy')
        if moving_average_checkbox.value:
            selected_models.append('Moving Average')
        
        print(f"Selected Models: {selected_models}")

button_model.on_click(on_button_clicked_model)

# Display all sections in a structured layout
display(widgets.VBox([
    widgets.HTML("<h2>Feature Selection</h2>"),
    grid,
    widgets.HBox([button_select_all_features, button_deselect_all_features]),
    button_features,
    output_features,
    widgets.HTML("<h2>Hyperparameter Selection</h2>"),
    hyperparameter_selection,
    button_hyperparams,
    output_hyperparams,
    widgets.HTML("<h2>Model Selection</h2>"),
    model_selection,
    widgets.HBox([button_select_all_models, button_deselect_all_models]),
    button_model,
    output_model
]))

In [20]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

def interactive_selection():
    # Define all possible features
    all_features = ['rate_level_1', 'rate_level_2', 'days_to_end_of_month', 'days_to_ECB_meeting', 'days_to_Fed_meeting', 'ois_sofr_rate',
                    'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15']

    # Function to split list into chunks
    def chunk_list(lst, n):
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    # Split the features into 3 columns
    num_columns = 3
    chunks = list(chunk_list(all_features, len(all_features) // num_columns + 1))

    # Create checkboxes for each feature
    checkboxes = [[widgets.Checkbox(value=True, description=feature) for feature in chunk] for chunk in chunks]

    # Create a grid layout for the checkboxes
    grid = widgets.GridBox([item for sublist in checkboxes for item in sublist],
                           layout=widgets.Layout(grid_template_columns=f'repeat({num_columns}, 200px)'))

    # Function to get selected features
    def get_selected_features():
        return [checkbox.description for sublist in checkboxes for checkbox in sublist if checkbox.value]

    # Add buttons to select/deselect all features
    button_select_all_features = widgets.Button(description="Select All Features")
    button_deselect_all_features = widgets.Button(description="Deselect All Features")

    def on_button_select_all_features(b):
        for sublist in checkboxes:
            for checkbox in sublist:
                checkbox.value = True

    def on_button_deselect_all_features(b):
        for sublist in checkboxes:
            for checkbox in sublist:
                checkbox.value = False

    button_select_all_features.on_click(on_button_select_all_features)
    button_deselect_all_features.on_click(on_button_deselect_all_features)

    # Add a button to confirm feature selection
    button_features = widgets.Button(description="Confirm Features")
    output_features = widgets.Output()

    # Global variable to store selected features
    selected_features = []

    def on_button_clicked_features(b):
        nonlocal selected_features
        with output_features:
            selected_features = get_selected_features()
            output_features.clear_output()
            print("Selected features:", selected_features)

    button_features.on_click(on_button_clicked_features)

    # Create widgets for hyperparameters with adjusted layout
    layout = widgets.Layout(width='400px')  # Adjust the width of the description area

    learning_rate_slider = widgets.FloatSlider(
        value=0.001,
        min=0.0001,
        max=0.01,
        step=0.00001,  # Increase precision
        description='Learning Rate:',
        continuous_update=False,
        readout_format='.5f',  # Display more decimal places
        style={'description_width': 'initial'},
        layout=layout
    )

    n_epochs_slider = widgets.IntSlider(
        value=50,
        min=10,
        max=200,
        step=10,
        description='Epochs:',
        continuous_update=False,
        style={'description_width': 'initial'},
        layout=layout
    )

    hidden_size_slider = widgets.IntSlider(
        value=128,
        min=32,
        max=512,
        step=32,
        description='Hidden Size:',
        continuous_update=False,
        style={'description_width': 'initial'},
        layout=layout
    )

    dropout_slider = widgets.FloatSlider(
        value=0.3,
        min=0.0,
        max=0.5,
        step=0.1,
        description='Dropout Rate:',
        continuous_update=False,
        style={'description_width': 'initial'},
        layout=layout
    )

    # Display the widgets for hyperparameters
    hyperparameter_selection = widgets.VBox([
        learning_rate_slider,
        n_epochs_slider,
        hidden_size_slider,
        dropout_slider
    ])

    # Add a button to confirm hyperparameters
    button_hyperparams = widgets.Button(description="Confirm Hyperparameters")
    output_hyperparams = widgets.Output()

    # Global variable to store selected hyperparameters
    selected_hyperparameters = {}

    def on_button_clicked_hyperparams(b):
        nonlocal selected_hyperparameters
        with output_hyperparams:
            selected_hyperparameters = {
                'learning_rate': learning_rate_slider.value,
                'n_epochs': n_epochs_slider.value,
                'hidden_size': hidden_size_slider.value,
                'dropout': dropout_slider.value
            }
            output_hyperparams.clear_output()
            print(f"Selected Hyperparameters:\nLearning Rate: {selected_hyperparameters['learning_rate']:.5f}\nEpochs: {selected_hyperparameters['n_epochs']}\nHidden Size: {selected_hyperparameters['hidden_size']}\nDropout Rate: {selected_hyperparameters['dropout']}")

    button_hyperparams.on_click(on_button_clicked_hyperparams)

    # Create checkboxes for model selection
    lstm_checkbox = widgets.Checkbox(value=False, description='LSTM', style={'description_width': 'initial'}, layout=layout)
    momentum_checkbox = widgets.Checkbox(value=False, description='Momentum Strategy', style={'description_width': 'initial'}, layout=layout)
    moving_average_checkbox = widgets.Checkbox(value=False, description='Moving Average', style={'description_width': 'initial'}, layout=layout)

    # Create a list of model checkboxes
    model_checkboxes = [lstm_checkbox, momentum_checkbox, moving_average_checkbox]

    # Add buttons to select/deselect all models
    button_select_all_models = widgets.Button(description="Select All Models")
    button_deselect_all_models = widgets.Button(description="Deselect All Models")

    def on_button_select_all_models(b):
        for checkbox in model_checkboxes:
            checkbox.value = True

    def on_button_deselect_all_models(b):
        for checkbox in model_checkboxes:
            checkbox.value = False

    button_select_all_models.on_click(on_button_select_all_models)
    button_deselect_all_models.on_click(on_button_deselect_all_models)

    # Display the model selection widgets
    model_selection = widgets.VBox(model_checkboxes)

    # Add a button to confirm model selection
    button_model = widgets.Button(description="Confirm Model")
    output_model = widgets.Output()

    # Global variable to store selected models
    selected_models = []

    def on_button_clicked_model(b):
        nonlocal selected_models
        with output_model:
            output_model.clear_output()
            selected_models = [checkbox.description for checkbox in model_checkboxes if checkbox.value]
            print(f"Selected Models: {selected_models}")

    button_model.on_click(on_button_clicked_model)

    # Display all sections in a structured layout
    display(widgets.VBox([
        widgets.HTML("<h2>Feature Selection</h2>"),
        grid,
        widgets.HBox([button_select_all_features, button_deselect_all_features]),
        button_features,
        output_features,
        widgets.HTML("<h2>Hyperparameter Selection</h2>"),
        hyperparameter_selection,
        button_hyperparams,
        output_hyperparams,
        widgets.HTML("<h2>Model Selection</h2>"),
        model_selection,
        widgets.HBox([button_select_all_models, button_deselect_all_models]),
        button_model,
        output_model
    ]))

    return selected_features, selected_hyperparameters, selected_models

# Call the function and get the selected values
selected_features, selected_hyperparameters, selected_models = interactive_selection()

# Function to display the selected values
def display_selected_values():
    print("Selected Features:", selected_features)
    print("Selected Hyperparameters:", selected_hyperparameters)
    print("Selected Models:", selected_models)

# Display the selected values in the next cell
display_selected_values()

Selected Features: []
Selected Hyperparameters: {}
Selected Models: []


In [31]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# Global variables to store selected values
selected_features = []
selected_hyperparameters = {}
selected_models = []

def interactive_selection():
    global selected_features, selected_hyperparameters, selected_models

    # Define all possible features
    all_features = ['rate_level_1', 'rate_level_2', 'days_to_end_of_month', 'days_to_ECB_meeting', 'days_to_Fed_meeting', 'ois_sofr_rate',
                    'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15']

    # Function to split list into chunks
    def chunk_list(lst, n):
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    # Split the features into 3 columns
    num_columns = 3
    chunks = list(chunk_list(all_features, len(all_features) // num_columns + 1))

    # Create checkboxes for each feature
    checkboxes = [[widgets.Checkbox(value=True, description=feature) for feature in chunk] for chunk in chunks]

    # Create a grid layout for the checkboxes
    grid = widgets.GridBox([item for sublist in checkboxes for item in sublist],
                           layout=widgets.Layout(grid_template_columns=f'repeat({num_columns}, 200px)'))

    # Function to get selected features
    def get_selected_features():
        return [checkbox.description for sublist in checkboxes for checkbox in sublist if checkbox.value]

    # Add buttons to select/deselect all features
    button_select_all_features = widgets.Button(description="Select All Features")
    button_deselect_all_features = widgets.Button(description="Deselect All Features")

    def on_button_select_all_features(b):
        for sublist in checkboxes:
            for checkbox in sublist:
                checkbox.value = True

    def on_button_deselect_all_features(b):
        for sublist in checkboxes:
            for checkbox in sublist:
                checkbox.value = False

    button_select_all_features.on_click(on_button_select_all_features)
    button_deselect_all_features.on_click(on_button_deselect_all_features)

    # Add a button to confirm feature selection
    button_features = widgets.Button(description="Confirm Features")
    output_features = widgets.Output()

    def on_button_clicked_features(b):
        global selected_features
        with output_features:
            selected_features = get_selected_features()
            output_features.clear_output()
            print("Selected features:", selected_features)

    button_features.on_click(on_button_clicked_features)

    # Create widgets for hyperparameters with adjusted layout
    layout = widgets.Layout(width='400px')  # Adjust the width of the description area

    learning_rate_slider = widgets.FloatSlider(
        value=0.001,
        min=0.0001,
        max=0.01,
        step=0.00001,  # Increase precision
        description='Learning Rate:',
        continuous_update=False,
        readout_format='.5f',  # Display more decimal places
        style={'description_width': 'initial'},
        layout=layout
    )

    n_epochs_slider = widgets.IntSlider(
        value=50,
        min=10,
        max=200,
        step=10,
        description='Epochs:',
        continuous_update=False,
        style={'description_width': 'initial'},
        layout=layout
    )

    hidden_size_slider = widgets.IntSlider(
        value=128,
        min=32,
        max=512,
        step=32,
        description='Hidden Size:',
        continuous_update=False,
        style={'description_width': 'initial'},
        layout=layout
    )

    dropout_slider = widgets.FloatSlider(
        value=0.3,
        min=0.0,
        max=0.5,
        step=0.1,
        description='Dropout Rate:',
        continuous_update=False,
        style={'description_width': 'initial'},
        layout=layout
    )

    # Display the widgets for hyperparameters
    hyperparameter_selection = widgets.VBox([
        learning_rate_slider,
        n_epochs_slider,
        hidden_size_slider,
        dropout_slider
    ])

    # Add a button to confirm hyperparameters
    button_hyperparams = widgets.Button(description="Confirm Hyperparameters")
    output_hyperparams = widgets.Output()

    def on_button_clicked_hyperparams(b):
        global selected_hyperparameters
        with output_hyperparams:
            selected_hyperparameters = {
                'learning_rate': learning_rate_slider.value,
                'n_epochs': n_epochs_slider.value,
                'hidden_size': hidden_size_slider.value,
                'dropout': dropout_slider.value
            }
            output_hyperparams.clear_output()
            print(f"Selected Hyperparameters:\nLearning Rate: {selected_hyperparameters['learning_rate']:.5f}\nEpochs: {selected_hyperparameters['n_epochs']}\nHidden Size: {selected_hyperparameters['hidden_size']}\nDropout Rate: {selected_hyperparameters['dropout']}")

    button_hyperparams.on_click(on_button_clicked_hyperparams)

    # Create checkboxes for model selection
    lstm_checkbox = widgets.Checkbox(value=False, description='LSTM', style={'description_width': 'initial'}, layout=layout)
    momentum_checkbox = widgets.Checkbox(value=False, description='Momentum Strategy', style={'description_width': 'initial'}, layout=layout)
    moving_average_checkbox = widgets.Checkbox(value=False, description='Moving Average', style={'description_width': 'initial'}, layout=layout)

    # Create a list of model checkboxes
    model_checkboxes = [lstm_checkbox, momentum_checkbox, moving_average_checkbox]

    # Add buttons to select/deselect all models
    button_select_all_models = widgets.Button(description="Select All Models")
    button_deselect_all_models = widgets.Button(description="Deselect All Models")

    def on_button_select_all_models(b):
        for checkbox in model_checkboxes:
            checkbox.value = True

    def on_button_deselect_all_models(b):
        for checkbox in model_checkboxes:
            checkbox.value = False

    button_select_all_models.on_click(on_button_select_all_models)
    button_deselect_all_models.on_click(on_button_deselect_all_models)

    # Display the model selection widgets
    model_selection = widgets.VBox(model_checkboxes)

    # Add a button to confirm model selection
    button_model = widgets.Button(description="Confirm Model")
    output_model = widgets.Output()

    def on_button_clicked_model(b):
        global selected_models
        with output_model:
            output_model.clear_output()
            selected_models = [checkbox.description for checkbox in model_checkboxes if checkbox.value]
            print(f"Selected Models: {selected_models}")

    button_model.on_click(on_button_clicked_model)

    # Display all sections in a structured layout
    display(widgets.VBox([
        widgets.HTML("<h2>Feature Selection</h2>"),
        grid,
        widgets.HBox([button_select_all_features, button_deselect_all_features]),
        button_features,
        output_features,
        widgets.HTML("<h2>Hyperparameter Selection</h2>"),
        hyperparameter_selection,
        button_hyperparams,
        output_hyperparams,
        widgets.HTML("<h2>Model Selection</h2>"),
        model_selection,
        widgets.HBox([button_select_all_models, button_deselect_all_models]),
        button_model,
        output_model
    ]))

# Call the function to display the widgets
interactive_selection()

In [3]:
import pandas as pd

# Sample data for demonstration
data1 = {
    'date': pd.date_range(start='2021-01-01', end='2021-01-10'),
    'instrument1': range(10),
    'instrument2': range(10, 20)
}

data2 = {
    'date': pd.date_range(start='2021-01-11', end='2021-01-20'),
    'instrument3': range(20, 30),
    'instrument4': range(30, 40)
}

# Create DataFrames
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# Set the date column as the index
df1.set_index('date', inplace=True)
df2.set_index('date', inplace=True)

# Rename the columns of df2 to match the corresponding instruments from df1
df2.rename(columns={'instrument3': 'instrument1', 'instrument4': 'instrument2'}, inplace=True)

# Concatenate the DataFrames
final_df = pd.concat([df1, df2])

# Display the final DataFrame
# print(final_df)